## 20 뉴스 그룹 분류

In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all',random_state=156)

- 데이터 탐색

In [4]:
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
pd.Series(news_data.target).value_counts().sort_index()

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

In [6]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 훈련/테스트용 데이터 추출

In [7]:
train_news = fetch_20newsgroups(subset='train', random_state=150, remove=('headers','footers','quotes'))
X_train = train_news.data
y_train = train_news.target

In [8]:
print(train_news.data[0])


   >Thousands?  Tens of thousands?  Do some arithmetic, please...  Skipjack
   >has 2^80 possible keys.  Let's assume a brute-force engine like that
   >hypothesized for DES:  1 microsecond per trial, 1 million chips.  That's
   >10^12 trials per second, or about 38,000 years for 2^80 trials.  Well,
   >maybe they can get chips running at one trial per nanosecond, and build
   >a machine with 10 million chips.  Sure -- only 3.8 years for each solution.

   But there is a MUCH more pernicious problem with the scheme as
proposed.  Building a brute force machine to test 2^40 possible keys
if you have the other half from one escrow agent is EASY.  (One chip,
one test per microsecond gives you one break every two weeks, and that
break gives you all messages involving that phone.)

   The XOR scheme so that the files from one escrow agent gives you
nothing is an improvement, but notice that XORing with (truely random)
bit strings allows for an arbitrary number of escrow agents.  Using +
for

In [9]:
train_news.target[0], train_news.target_names[train_news.target[0]]

(11, 'sci.crypt')

In [10]:
test_news = fetch_20newsgroups(subset='test', random_state=156, remove=('headers','footers','quotes'))
X_test = test_news.data
y_test = test_news.target

In [11]:
len(X_train), len(X_test)

(11314, 7532)

### 피쳐 벡터화 변환과 머신러닝 모델 학습/예측/평가
- Case 1. CountVectorizer + LogisticRegression

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [13]:
X_train_cv.shape, X_test_cv.shape

((11314, 101631), (7532, 101631))

In [16]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_cv, y_train)

C:\Users\kimee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=300)

In [15]:
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [18]:
from sklearn.metrics import accuracy_score
pred=lr.predict(X_test_cv)
accuracy_score(y_test,pred)

0.5962559745087627

In [19]:
y_test[:5], pred[:5]

(array([ 4, 11,  1,  7,  8]), array([ 4, 11,  6,  7,  8]))

- Case 2. TfidVectorizer + LogisticRegression

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer()
tvect.fit(X_train)
X_train_tf = tvect.transform(X_train)
X_test_tf = tvect.transform(X_test)

In [21]:
X_train_tf.shape, X_test_tf.shape

((11314, 101631), (7532, 101631))

In [24]:
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_tf, y_train)

LogisticRegression(max_iter=300)

In [25]:
pred = lr.predict(X_test_tf)
accuracy_score(y_test,pred)

0.6736590546999469

In [26]:
y_test[:5], pred[:5]

(array([ 4, 11,  1,  7,  8]), array([5, 1, 1, 7, 8]))

- Case 3. stop_words filtering, ngram_range=(1,2), max_df=300

In [27]:
tvect2 = TfidfVectorizer(ngram_range=(1,2), max_df=300, stop_words='english')
tvect2.fit(X_train)
X_train_tf2 = tvect2.transform(X_train)
X_test_tf2 = tvect2.transform(X_test)

In [29]:
X_train_tf2.shape, X_test_tf2.shape

((11314, 943453), (7532, 943453))

In [28]:
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_tf2, y_train)
pred2 = lr.predict(X_test_tf2)
accuracy_score(y_test, pred2)

0.6922464152947424

- Case 4. Case 3에서 LogisticRegression C값을 10으로 

In [30]:
lr = LogisticRegression(max_iter=300, C=10)
lr.fit(X_train_tf2, y_train)
pred2 = lr.predict(X_test_tf2)
accuracy_score(y_test, pred2)

0.7012745618693574

### Pipeline과 GridSearchCV를 통한 하이퍼 파라미터 튜닝

In [33]:
from sklearn.pipeline import Pipeline

In [34]:
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(stop_words='english')),
    ('lr', LogisticRegression())
])

In [35]:
params = {
    'tvect__ngram_range':[(1,1),(1,2)],
    'tvect__max_df':[300,700],
    'lr__C':[1,10]
}

In [36]:
from sklearn.model_selection import GridSearchCV

grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1, n_jobs=-1
    )

In [37]:
%time grid_pipe.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [ ]:
print(grid_pipe.best_score_)
print(grid_pipe.best_params_)
pred = grid_pipe.best_estimator_(X_test)
accuracy_score(y_test, pred)